In [71]:
import pickle
import os
from pathlib import Path
from algo_2 import *
#0:144 result done
with open('output 0 to 144','rb') as f:
     output_0_144 = pickle.load(f)
len(output_0_144)

24

In [72]:
#1973:2117 result done
with open('output 1973 to 2116','rb') as f:
     output_2116_1973 = pickle.load(f)
len(output_2116_1973)

24

In [73]:
#1781:1973 result  done
with open('output 1781 to 1972','rb') as f:
     output_1781_1972 = pickle.load(f)
len(output_1781_1972)

32

In [74]:
#1685:1781 result 
with open('output 1685 to 1780','rb') as f:
     output_1685_1780 = pickle.load(f)
len(output_1685_1780)

11

In [ ]:
# 1589:1685 result done

In [75]:
#1445:1589 result done
with open('output 1445 to 1588','rb') as f:
     output_1445_1588 = pickle.load(f)
len(output_1445_1588)

22

In [ ]:
#1301:1445 result

In [76]:
#1157:1301 result done
with open('output 1157 to 1300','rb') as f:
     output_1157_1300 = pickle.load(f)
len(output_1157_1300)


23

In [ ]:
#677:1157 result

In [ ]:
#677:677 result

In [77]:
# List all files in the directory that start with "output "
directory = os.getcwd()
pickle_files = [f for f in os.listdir(directory) if f.startswith("output ")]

# Initialize a dictionary to store the contents of each pickle file
pickle_contents = []

# Loop through each file, read its content, and store it in the dictionary
for file_name in pickle_files:
    file_path = os.path.join(directory, file_name)
    with open(file_path, 'rb') as file:
        pickle_contents.extend(pickle.load(file))

# pickle_contents.keys()
pickle_contents.sort(key=lambda x: x[0][1], reverse=True)
all_tickers = [i[0][0]["ticker"] for i in pickle_contents]
len(all_tickers)

148

In [94]:
finer_pick = []
a = []
b=[]
c=[]
finer_finer_pick = []
for i in pickle_contents:
    for j in range(len(i)):
        stock = algo2(**i[j][0])
        # eval(stock, opt=False)
        maximum_drawdown, ROI, win_loss_ratio = eval(stock, opt=True)
        buy_orders = []
        sell_orders = []
        current_order = None
        money = 100000
        short = None
        long = None

        for index, row in stock.iterrows():
            if row['order'] > 0:
                if current_order is None and short is None:
                    current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': stock.loc[index -
                                                                                                            1, 'close'], "Shares Purchased": money/stock.loc[index - 1, 'close'], "long/short": "long"}
                    long = True
                elif current_order is not None and short is not None:
                    current_order['Sell Date'] = row['date']
                    current_order['Sell Price'] = row['close']
                    current_order["Price Change"] = current_order["Sell Price"] - \
                        current_order["Buy Price"]
                    current_order["Profit"] = current_order["Price Change"] * \
                        current_order["Shares Purchased"] * -1
                    current_order["Money After Order"] = current_order["Shares Purchased"] * \
                        current_order['Buy Price'] + current_order["Profit"]
                    money = current_order["Money After Order"]
                    sell_orders.append(current_order)
                    short = None
                    current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': row[
                        'close'], "Shares Purchased": money/row['close'], "long/short": "long"}
                    long = True
                else:
                    continue
            elif row['order'] < 0:
                if current_order is None and long is None:
                    current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': stock.loc[index -
                                                                                                            1, 'close'], "Shares Purchased": money/stock.loc[index - 1, 'close'], "long/short": "short"}
                    short = True
                elif current_order is not None and long is not None:
                    current_order['Sell Date'] = row['date']
                    current_order['Sell Price'] = row['close']
                    current_order["Price Change"] = current_order["Sell Price"] - \
                        current_order["Buy Price"]
                    current_order["Profit"] = current_order["Price Change"] * \
                        current_order["Shares Purchased"]
                    current_order["Money After Order"] = current_order["Shares Purchased"] * \
                        current_order['Buy Price'] + current_order["Profit"]
                    money = current_order["Money After Order"]
                    sell_orders.append(current_order)
                    long = None
                    current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': row[
                        'close'], "Shares Purchased": money/row['close'], "long/short": "short"}
                    short = True
                else:
                    continue
            elif row['order'] == 0:
                if current_order is not None:
                    current_order['Sell Date'] = row['date']
                    current_order['Sell Price'] = row['close']
                    current_order["Price Change"] = current_order["Sell Price"] - \
                        current_order["Buy Price"]
                    if current_order['long/short'] == "short":
                        current_order["Profit"] = current_order["Price Change"] * \
                            current_order["Shares Purchased"] * -1
                    elif current_order['long/short'] == "long":
                        current_order["Profit"] = current_order["Price Change"] * \
                            current_order["Shares Purchased"]
                    current_order["Money After Order"] = current_order["Shares Purchased"] * \
                        current_order['Buy Price'] + current_order["Profit"]
                    money = current_order["Money After Order"]
                    sell_orders.append(current_order)
                    current_order = None
                    long = None
                    short = None

        # Create a new DataFrame for buy and sell orders
        orders_df = pd.DataFrame(buy_orders + sell_orders)
        if len(orders_df) >= 15:
            if i[j][0]["ticker"] not in a:
                a.append(i[j][0]["ticker"])
            if ROI >= 2.2 and maximum_drawdown <= 0.12 and win_loss_ratio >= 0.75:
                if i[j][0]["ticker"] not in finer_pick:
                    finer_pick.append(i[j][0]["ticker"])
            if ROI >= 2.2 and maximum_drawdown <= 0.10 and win_loss_ratio >= 0.78:
                if i[j][0]["ticker"] not in b:
                    b.append(i[j][0]["ticker"])
            if ROI >= 2.2 and maximum_drawdown <= 0.15 and win_loss_ratio >= 0.8:
                if i[j][0]["ticker"] not in c:
                    c.append(i[j][0]["ticker"])
            if ROI >=5 or maximum_drawdown <= 0.06 or win_loss_ratio >= 0.85:
                if i[j][0]["ticker"] not in finer_finer_pick:
                    finer_finer_pick.append(i[j][0]["ticker"])

In [95]:
data_path = Path(os.getcwd())
path = data_path / 'stock price data'
finer_pick = set(finer_pick)
ticker_all = pd.read_csv(path/"finer_data_complete_stock.csv")
ticker_finer = ticker_all[ticker_all["Symbol"].isin(finer_pick)]
ticker_finer.groupby("Sector")["Sector"].count()

Sector
Basic Materials            6
Communication Services     2
Consumer Cyclical         11
Consumer Defensive         4
Energy                     1
Financial Services         4
Healthcare                 3
Industrials                5
Technology                 6
Utilities                  1
Name: Sector, dtype: int64

In [96]:
ticker_a = ticker_all[ticker_all["Symbol"].isin(a)]
ticker_a.groupby("Sector")["Sector"].count()

Sector
Basic Materials            9
Communication Services     5
Consumer Cyclical         26
Consumer Defensive         6
Energy                     8
Financial Services        17
Healthcare                12
Industrials               22
Real Estate                3
Technology                32
Utilities                  3
Name: Sector, dtype: int64

In [97]:
ticker_b = ticker_all[ticker_all["Symbol"].isin(b)]
ticker_b.groupby("Sector")["Sector"].count()

Sector
Consumer Cyclical     9
Consumer Defensive    3
Financial Services    2
Healthcare            1
Industrials           1
Technology            4
Utilities             1
Name: Sector, dtype: int64

In [98]:
ticker_c = ticker_all[ticker_all["Symbol"].isin(c)]
ticker_c.groupby("Sector")["Sector"].count()

Sector
Basic Materials           3
Communication Services    1
Consumer Cyclical         7
Consumer Defensive        3
Energy                    2
Financial Services        5
Healthcare                3
Industrials               2
Real Estate               1
Technology                8
Utilities                 1
Name: Sector, dtype: int64

In [99]:
finer_finer_pick = set(finer_finer_pick)
ticker_finer_finer = ticker_all[ticker_all["Symbol"].isin(finer_finer_pick)]
ticker_finer_finer.groupby("Sector")["Sector"].count()

Sector
Basic Materials            4
Communication Services     1
Consumer Cyclical          6
Consumer Defensive         2
Energy                     2
Financial Services         9
Healthcare                 5
Industrials                4
Technology                13
Utilities                  1
Name: Sector, dtype: int64

In [100]:
best_dict_finer = {}
best_dict_finer_finer = {}
for i in pickle_contents:
    if i[0][0]["ticker"] in finer_pick:
        best_dict_finer[i[0][0]["ticker"]] = i[0][0]
    if i[0][0]["ticker"] in finer_finer_pick:
        best_dict_finer_finer[i[0][0]["ticker"]] = i[0][0]

best_dict_finer = pd.DataFrame(best_dict_finer).T
best_dict_finer_finer = pd.DataFrame(best_dict_finer_finer).T
best_dict_finer.describe()

,atr_len,macd_fastLen,macd_slowLen,macd_signalSmooth,macd_peakLen,gain_ratio,loss_ratio,peak2_len,peak3_len,ticker
count,43,43,43,43,43,43,43,43,43,43
unique,3,3,2,3,2,2,2,1,2,43
top,11,11,32,7,3,1,2,20,50,AMEH
freq,37,22,24,24,42,35,33,43,29,1


In [101]:
best_dict_finer_finer.describe()

,atr_len,macd_fastLen,macd_slowLen,macd_signalSmooth,macd_peakLen,gain_ratio,loss_ratio,peak2_len,peak3_len,ticker
count,47,47,47,47,47,47,47,47,47,47
unique,3,3,2,3,2,2,2,1,2,47
top,11,13,34,7,3,1,2,20,50,SKE
freq,36,19,25,18,45,39,26,47,33,1
